JUPYTER NOTEBOOK FOR FELINE DATA ANALYSIS

In [1]:

import numpy as np
import pandas as pd
%matplotlib inline

import os
os.getcwd()

Cranial = pd.read_csv('/Users/88firefly/Feline-Bones-Final-Project/data/Felidae Cranial Data.csv',delimiter=',')
Forelimb= pd.read_csv('/Users/88firefly/Feline-Bones-Final-Project/data/Felidae and Canidae postcranial database.csv',delimiter=',')

Forelimb_Data=Forelimb.values[:261,:] #cut out dogs

Forelimb_genusID=Forelimb_Data[:,2]
Forelimb_speciesID=Forelimb_Data[:,3]
Forelimb_sexID=Forelimb_Data[:,4]

Cranial_Data=Cranial.values

Cranial_genusID=Cranial_Data[:,2]
Cranial_speciesID=Cranial_Data[:,3]
Cranial_sexID=Cranial_Data[:,4]

print("The Cranial dataset has", np.unique(Cranial[['Species']]).shape, "unique species, and", Cranial.shape, "samples.")
print("The Forelimb dataset has", np.unique(Forelimb[['Species']]).shape, "unique species, and", Forelimb.shape, "samples.")


The Cranial dataset has (77,) unique species, and (338, 52) samples.
The Forelimb dataset has (95,) unique species, and (323, 61) samples.


In [2]:
Forelimb.shape

(323, 61)

In [22]:
ComboData_All=Cranial.merge(Forelimb,how='outer', on=None, left_on='Catalog no.', right_on='Catalog no.')
ComboData_Shared=Cranial.merge(Forelimb,how='inner', on=None, left_on='Catalog no.', right_on='Catalog no.')

KeyError: 'Catalog no., Museum'

In [4]:
ComboShared_Data=ComboData_Shared.values
ComboAll_Data=ComboData_All.values

In [33]:
from sklearn.preprocessing import Imputer
print(ComboShared_Data[-1,:])  #52-57 unnecessary
print(Cranial.shape)
print(Forelimb.shape)

ComboShared_NumData=np.delete(ComboShared_Data,np.arange(52,58),axis=1)
print(ComboShared_NumData[-1,:])

['MVZ' '81547' 'Puma' 'concolor' 'unknown' 'U.S.'
 'Montana; Bitterroot river' 224.7 68.0 90.0 67.4 85.7 64.9 155.3 83.8 46.7
 79.1 24.2 59.0 74.6 42.3 93.8 6.5 5.2 14.8 12.3 10.0 15.4 12.3 22.9 64.7
 24.8 14.1 10.6 9.1 13.2 6.8 11.1 15.2 8.5 17.7 8.8 45.7 27.3 73.5 79.0
 144.9 27.6 15.0 25.4 14.6 42.3 'MVZ' 'Puma' 'concolor' 'unknown' 'U.S.'
 'Montana; Bitterroot river' 258.4 45.5 208.2 24.3 26.6 19.1 40.8 26.6 29.3
 18.4 248.7 62.8 23.0 63.1 39.0 42.3 83.4 14.3 18.5 16.8 15.2 10.6 90.9
 17.6 15.5 17.0 15.7 10.5 86.4 13.1 15.1 15.4 15.0 8.9 71.3 15.6 12.9 15.8
 14.8 9.1 35.2 15.0 12.4 10.8 9.0 9.2 28.0 13.0 11.4 11.7 8.6 8.8 25.7 19.4]
(338, 52)
(323, 61)
['MVZ' '81547' 'Puma' 'concolor' 'unknown' 'U.S.'
 'Montana; Bitterroot river' 224.7 68.0 90.0 67.4 85.7 64.9 155.3 83.8 46.7
 79.1 24.2 59.0 74.6 42.3 93.8 6.5 5.2 14.8 12.3 10.0 15.4 12.3 22.9 64.7
 24.8 14.1 10.6 9.1 13.2 6.8 11.1 15.2 8.5 17.7 8.8 45.7 27.3 73.5 79.0
 144.9 27.6 15.0 25.4 14.6 42.3 258.4 45.5 208.2 24.3 26.6 19.1

In [34]:
imp=Imputer(missing_values='NaN', strategy='mean', axis=0, copy=True)
Cranial_filled_data=imp.fit_transform(Cranial_Data[:,7:])
Forelimb_filled_data=imp.fit_transform(Forelimb_Data[:,7:])
imp.fit_transform(ComboShared_NumData[:,7:])

array([[ 178.2       ,   49.4       ,   63.5       , ...,    5.9       ,
          22.3       ,   13.8       ],
       [ 194.7       ,   34.        ,   79.5       , ...,    9.9       ,
          24.8       ,   15.        ],
       [ 194.7       ,   34.        ,   79.5       , ...,    3.51641791,
          24.6       ,   16.6       ],
       ..., 
       [ 182.1       ,   57.5       ,   71.6       , ...,    7.1       ,
          21.8       ,   15.1       ],
       [ 201.9       ,   60.8       ,   78.7       , ...,    6.5       ,
          23.2       ,   15.9       ],
       [ 224.7       ,   68.        ,   90.        , ...,    8.8       ,
          25.7       ,   19.4       ]])